# Introduction to TensorFlow

# Table of Contents

1. Graph and session
2. Tensor
3. Scopes and summaries
4. Placeholders
5. Saving and Restoring

# 0. Before everything, import!

In [2]:
import tensorflow as tf

# 1. Graph and session

A class for running TensorFlow operations.

A Session object encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated.

TensorFlow uses a dataflow graph to represent your computation in terms of the dependencies between individual operations. This leads to a low-level programming model in which you first define the dataflow graph, then create a TensorFlow session to run parts of the graph across a set of local and remote devices.

A TensorFlow Session for use in interactive contexts, such as a shell.

The only difference with a regular `Session` is that an `InteractiveSession` installs itself as the default session on construction. The methods `tf.Tensor.eval` and `tf.Operation.run` will use that session to run ops.

In [3]:
# Build a graph.
a = tf.constant(5.0)
b = tf.constant(6.0)
c = tf.multiply(a, b)

# First way
# Launch the graph in a session.
sess = tf.Session()

# Evaluate the tensor `c`.
print(sess.run(c))

# Using the `close()` method.
sess.close()

# Second way
# Using the context manager.
with tf.Session() as sess:
    
    # Evaluate the tensor `c`.
    print(sess.run(c))
    
# Third way
sess = tf.InteractiveSession()

# Evaluate the tensor `c`.
# If you have a Tensor t, calling t.eval() is equivalent 
# to calling tf.get_default_session().run(t)
print(c.eval())

# Using the `close()` method.
sess.close()

30.0
30.0
30.0


# 2. Tensor

A **tensor** is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes.

When writing a TensorFlow program, the main object you manipulate and pass around is the `tf.Tensor`. A `tf.Tensor` object represents a partially defined computation that will eventually produce a value. TensorFlow programs work by first building a graph of `tf.Tensor` objects, detailing how each tensor is computed based on the other available tensors and then by running parts of this graph to achieve the desired results.

A `tf.Tensor` has the following properties:

* a data type (float32, int32, or string, for example)
* a shape

The main ones are:

* `tf.Constant`
* `tf.Variable`
* `tf.Placeholder`
* `tf.SparseTensor`

The **rank** of a `tf.Tensor` object is its number of dimensions. Synonyms for rank include **order** or **degree** or **n-dimension**.

As the following list shows, each rank in TensorFlow corresponds to a different mathematical entity:

* 0	Scalar (magnitude only)
* 1	Vector (magnitude and direction)
* 2	Matrix (table of numbers)
* 3	3-Tensor (cube of numbers)
* n	n-Tensor (you get the idea)

## 2.1. Constants, Sequences, and Random Values

### Constant Value Tensors

* Constants are stored in the graph definition
* This makes loading graphs expensive when constants are big
* Only use constants for primitive types

`tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)`

* value: A constant value (or list) of output type dtype.
* dtype: The type of the elements of the resulting tensor.
* shape: Optional dimensions of resulting tensor.
* name: Optional name for the tensor.
* verify_shape: Boolean that enables verification of a shape of values

`tf.zeros(shape, dtype=tf.float32, name=None)`

Creates a tensor with all elements set to zero.

* shape: A list of integers, a tuple of integers, or a 1-D Tensor of type int32.
* dtype: The type of an element in the resulting Tensor.
* name: A name for the operation (optional).

`tf.ones(shape, dtype=tf.float32, name=None)`

Creates a tensor with all elements set to one.

* shape: A list of integers, a tuple of integers, or a 1-D Tensor of type int32.
* dtype: The type of an element in the resulting Tensor.
* name: A name for the operation (optional).

`tf.fill(dims, value, name=None)`

Creates a tensor filled with a scalar value.

* dims: A Tensor of type int32. 1-D. Represents the shape of the output tensor.
* value: A Tensor. 0-D (scalar). Value to fill the returned tensor.
* name: A name for the operation (optional).

In [6]:
a = tf.constant(1)
b = tf.constant(3)

x = tf.add(a, b)

with tf.Session() as sess:
    print('Result:', sess.run(x))

Result: 4


In [21]:
tf.reset_default_graph()

a = tf.constant(1)
b = tf.constant(3)
x = tf.add(a, b)

with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print('Result:', sess.run(x))

writer.close()

Result: 4


In [ ]:
tf.reset_default_graph()

a = tf.constant(1, name='a')
b = tf.constant(3, name='b')
x = tf.add(a, b, name='add')

with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print('Result:', sess.run(x))

writer.close()

In [9]:
tf.reset_default_graph()

a = tf.constant([[2, 2]], name='a')
b = tf.constant([[0, 1], [2, 3]], name='b')

# Po kordinatama!
x = tf.add(a, b, name='add')
y = tf.multiply(a, b, name='mul')

# Matrice
z = tf.matmul(a, b, name='matmul')

with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print('Result:', sess.run([x, y, z]))

writer.close()

Result: [array([[2, 3],
       [4, 5]], dtype=int32), array([[0, 2],
       [4, 6]], dtype=int32), array([[4, 8]], dtype=int32)]


In [10]:
tf.reset_default_graph()

a = tf.zeros([2, 3], tf.int32, name='a')
b = tf.zeros([2, 1], tf.int32, name='b')
c = tf.fill([2, 1, 3], 8.0)

with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print('Result:', sess.run([a, b, c]))

writer.close()

Result: [array([[0, 0, 0],
       [0, 0, 0]], dtype=int32), array([[0],
       [0]], dtype=int32), array([[[8., 8., 8.]],

       [[8., 8., 8.]]], dtype=float32)]


### Sequences

`tf.linspace(start, stop, num, name=None)`

Generates values in an interval.

A sequence of `num` evenly-spaced values are generated beginning at `start`. If `num > 1`, the values in the sequence increase by `stop - start / num - 1`, so that the last one is exactly stop.

* start: A Tensor. Must be one of the following types: float32, float64. First entry in the range.
* stop: A Tensor. Must have the same type as start. Last entry in the range.
* num: A Tensor. Must be one of the following types: int32, int64. Number of values to generate.
* name: A name for the operation (optional).

`tf.range(start, limit, delta=1, dtype=None, name='range')`

Creates a sequence of numbers.

Creates a sequence of numbers that begins at start and extends by increments of `delta` up to but not including `limit`.

* start: A 0-D Tensor (scalar). Acts as first entry in the range if limit is not None; otherwise, acts as range limit and first entry defaults to 0.
* limit: A 0-D Tensor (scalar). Upper limit of sequence, exclusive. If None, defaults to the value of start while the first entry of the range defaults to 0.
* delta: A 0-D Tensor (scalar). Number that increments start. Defaults to 1.
* dtype: The type of the elements of the resulting tensor.
* name: A name for the operation. Defaults to "range".

In [14]:
tf.reset_default_graph()

a = tf.linspace(10.0, 12.0, 3, name='a')
b = tf.range(1, 10, 2, name='b')

with tf.Session() as sess:
    print(sess.run([a, b]))

[array([10., 11., 12.], dtype=float32), array([1, 3, 5, 7, 9], dtype=int32)]


### Random Tensors

`tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)`

Outputs random values from a normal distribution.

* shape: A 1-D integer Tensor or Python array. The shape of the output tensor.
* mean: A 0-D Tensor or Python value of type dtype. The mean of the normal distribution.
* stddev: A 0-D Tensor or Python value of type dtype. The standard deviation of the normal distribution.
* dtype: The type of the output.
* seed: A Python integer. Used to create a random seed for the distribution. See tf.set_random_seed for behavior.
* name: A name for the operation (optional).

`tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)`

Outputs random values from a uniform distribution.

The generated values follow a uniform distribution in the range `[minval, maxval)`. The lower bound minval is included in the range, while the upper bound `maxval` is excluded.

For floats, the default range is `[0, 1)`. For ints, at least `maxval` must be specified explicitly.

In the integer case, the random integers are slightly biased unless `maxval - minval` is an exact power of two. The bias is small for values of `maxval - minval` significantly smaller than the range of the output (either `2**32` or `2**64`).

* shape: A 1-D integer Tensor or Python array. The shape of the output tensor.
* minval: A 0-D Tensor or Python value of type dtype. The lower bound on the range of random values to generate. Defaults to 0.
* maxval: A 0-D Tensor or Python value of type dtype. The upper bound on the range of random values to generate. Defaults to 1 if dtype is floating point.
* dtype: The type of the output: float16, float32, float64, int32, or int64.
* seed: A Python integer. Used to create a random seed for the distribution. See `tf.set_random_seed` for behavior.
* name: A name for the operation (optional).

Other usefull functions:

* `tf.truncated_normal`
* `tf.random_uniform`
* `tf.random_shuffle`
* `tf.random_crop`
* `tf.multinomial`
* `tf.random_gamma`

In [ ]:
# The graph-level seed
tf.set_random_seed(1)

In [15]:
norm = tf.random_normal([2, 3], mean=-1, stddev=4, seed=1)
unif = tf.random_uniform([1, 3], minval=0, maxval=10)

with tf.Session() as sess:
    print(sess.run(norm))
    print(sess.run(norm))
    print(sess.run(unif))
print()
with tf.Session() as sess:
    print(sess.run(norm))
    print(sess.run(norm))

[[ -4.2452726    4.938395    -0.7386825 ]
 [-10.770817    -0.60300636   1.3648973 ]]
[[-3.7036238   0.14663327  0.28635478]
 [ 3.849855   -4.2234287  -1.5339782 ]]
[[6.459399   2.722218   0.46063542]]

[[ -4.2452726    4.938395    -0.7386825 ]
 [-10.770817    -0.60300636   1.3648973 ]]
[[-3.7036238   0.14663327  0.28635478]
 [ 3.849855   -4.2234287  -1.5339782 ]]


## 2.2. Variables

Tensor Ranks, Shapes, and Types : https://www.tensorflow.org/versions/r0.12/resources/dims_types

A TensorFlow **variable** is the best way to represent shared, persistent state manipulated by your program.

Variables are manipulated via the tf.Variable class. A tf.Variable represents a tensor whose value can be changed by running ops on it. Unlike `tf.Tensor` objects, a `tf.Variable` exists outside the context of a single session.run call.

Internally, a `tf.Variable` stores a persistent tensor. Specific ops allow you to read and modify the values of this tensor. These modifications are visible across multiple `tf.Sessions`, so multiple workers can see the same values for a `tf.Variable`.

Each session maintains its own copy of variable.

In [16]:
# create variable a with scalar value
a = tf.Variable(2, name="scalar")

# create variable b as a vector
b = tf.Variable([2, 3], name="vector")

# create variable c as a 2x2 matrix
c = tf.Variable([[0, 1], [2, 3]], name="matrix")

# Initialize variable with zeros
d = tf.Variable(tf.zeros([2, 2]), name="zeros")

# Initialize variable with random uniform numbers
e = tf.Variable(tf.random_uniform([1, 3], minval=0, maxval=10), name="random_uniform")

# create variable W as 784 x 10 tensor, filled with zeros
W = tf.Variable(tf.zeros([784,10]))

# Initializing all variables at once
init = tf.global_variables_initializer()

# Initialize only a subset of variables
init_ab = tf.variables_initializer([a, b], name="init_ab")

# Initialize assign ops
a_new = a.assign(tf.add(a, 2))

with tf.Session() as sess:
    
    sess.run(init)
    
    print(sess.run([d]))
    
    print(sess.run([e]))
    
    # print(sess.run([a, c]))
    print(a.eval())
    sess.run(a_new)

    print(a.eval())

[array([[0., 0.],
       [0., 0.]], dtype=float32)]
[array([[1.7090285, 4.2903066, 3.5735536]], dtype=float32)]
2
4


# Placeholders

`tf.placeholder(dtype, shape=None, name=None)`

Inserts a placeholder for a tensor that will be always fed.

**Important**: This tensor will produce an error if evaluated. Its value must be fed using the `feed_dict` optional argument to `Session.run()`, `Tensor.eval()`, or `Operation.run()`.

* `dtype`: The type of elements in the tensor to be fed.
* `shape`: The shape of the tensor to be fed (optional). If the shape is not specified, you can feed a tensor of any shape.
* `name`: A name for the operation (optional).

Analogy:

Can define the function $f(x, y) = x*2 + y$ without knowing value of $x$ or $y$. $x, y$ are placeholders for the actual values. Can assemble the graph first without knowing the values needed for computation.


In [18]:
tf.reset_default_graph()

x = tf.placeholder(shape=(1, 3), dtype=tf.float32, name="x")
y = tf.matmul(x, tf.transpose(x), name="y")
z = tf.multiply(x, 2, name="z")

with tf.Session() as sess:
    
    # ERROR: will fail because x was not fed.
    # print(sess.run(y))
    
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run([y, z], feed_dict={x: [[1, 2, 3]]}))  # Will succeed.
    
    # Feed multiple data points in placeholder
    for i in range(10):
        print(sess.run(z, {x : [[i, i, i]]}))
    
writer.close()

[array([[14.]], dtype=float32), array([[2., 4., 6.]], dtype=float32)]
[[0. 0. 0.]]
[[2. 2. 2.]]
[[4. 4. 4.]]
[[6. 6. 6.]]
[[8. 8. 8.]]
[[10. 10. 10.]]
[[12. 12. 12.]]
[[14. 14. 14.]]
[[16. 16. 16.]]
[[18. 18. 18.]]


# Saving and Restoring

In [19]:
tf.reset_default_graph()

# Create some variables.
a = tf.Variable(tf.random_normal([1]), name="a")
b = tf.Variable(tf.random_normal([1]), name="b")

x = tf.placeholder(shape=[1], dtype=tf.float32, name="x")

y = tf.add(tf.multiply(a, x), b, name="y")

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(init_op)
    
    print(sess.run([a, b]))
    print(sess.run(y, feed_dict={x: [2.0]}))
    
    sess.run(a.assign_add([2.0]))
    sess.run(b.assign_sub([1.0]))
    
    print(sess.run([a, b]))
    print(sess.run(y, feed_dict={x: [2.0]}))
    
    saver.save(sess, './models/liner_function_model')
    print('Model saved')

[array([0.16410099], dtype=float32), array([-0.3397109], dtype=float32)]
[-0.01150891]
[array([2.164101], dtype=float32), array([-1.339711], dtype=float32)]
[2.9884908]
Model saved


In [20]:
with tf.Session() as sess:
    model = tf.train.import_meta_graph('./models/liner_function_model.meta')
    model.restore(sess, tf.train.latest_checkpoint('./models/'))
    graph = tf.get_default_graph()
    # x = graph.get_tensor_by_name("x:0")
    # y = graph.get_tensor_by_name("y:0")
    print(sess.run("y:0", feed_dict={"x:0": [2.0]}))

INFO:tensorflow:Restoring parameters from ./models/liner_function_model
[2.9884908]


# Resources

1. [TensorFlow: Programmer's Guide][1]
2. [CS 20: Tensorflow for Deep Learning Research][2]

[1]: https://www.tensorflow.org/programmers_guide/
[2]: http://web.stanford.edu/class/cs20si/